In [1]:
from pathlib import Path
from thermophysicalProperties import Database
import matplotlib.pyplot as plt
import numpy as np
from frozendict import frozendict
import pint
ureg = pint.UnitRegistry(auto_reduce_dimensions=True)
from uncertainties import ufloat
import os
from numdifftools import Derivative
from scipy.optimize import minimize
import warnings
import sys

warnings.filterwarnings("ignore")

# Get the script name from sys.argv
script_name = sys.argv[0]

# Get the absolute path of the script
script_dir = Path(os.path.abspath(script_name)).resolve().parent

mstdb_tp_path = Path('../../mstdb-tp/Molten_Salt_Thermophysical_Properties.csv')
mstdb_tp_rk_path = Path('../../mstdb-tp/Molten_Salt_Thermophysical_Properties_2.1.0_RK.csv')

db = Database(mstdb_tp_path, mstdb_tp_rk_path)

# Coolant Salt Optimization
In the literature, there seems to be a consensus that a fluoride-based coolant salt is superior from the standpoint of thermophsyical properties, so it was chosen for this project. The following endmembers are under consideration (and their composition will be optimized in the following section)
1. $\text{NaF}$
2. $\text{KF}$
3. $\text{ZrF}_4$
4. $\text{CaF}_2$

Notably, $\text{LiF}$ and $\text{BeF}_2$ were excluded because of the prototypical issues caused by tritium breeding in lithium, and the relatively high chemical toxicity of beryllium. The following criteria where considered for optimization
1. High heat capacity
2. Large volumetric expansion coefficient 
   1. I.e. $\beta = - \frac{1}{\rho}\big( \frac{\partial \rho}{\partial T}\big)_p$
3. Low viscosity

This is inherently a multiobjective problem, so we must have a method for evaluating each salt based on its thermophysical properties. To simplify the analysis, consider the following expression for the heat transfer rate
\begin{equation}
   \dot{q}_{cool} = 2\pi r L \overline{h} (\overline{T}_s - \overline{T}_\infty) = 2\pi r L \overline{h}\Delta T\nonumber
\end{equation}
where $L$ is the core height, and $2\pi rL$ represents the heated surface area of a single fuel rod of outer radius $r$. $\overline{h}$ is the heat transfer coefficient, and $\overline{T}_s$ and $\overline{T}_\infty$ are the average surface temperature of the fuel rod and the average ambient coolant temperature respectively. The heat transfer rate increases if the surface temperature increases, however this is constrained by material integrity, in addition, increasing the core height or the fuel rod radius will also increase the heat transfer rate, but doing so will lead to higher temperatures within the fuel rod, and these changes are also constrained. The only variable that can be nontrivially changed is the heat transfer coefficient, which depends on the thermophysical properties of the coolant. It is for this reason that we fix $r=4$mm, $L=5$m, and consider a (limiting) $\Delta T = 200$K (and $T_\infty = 1000$K), however it is the hope that these values should not affect the optimum salt composition, otherwise our minimum is conditional on the assumed operating parameters.
  
So, for a simplfied analysis, it is sufficient to optimize the heat transfer coefficient $h$, which is given in terms of the Nusselt number via
\begin{equation}
   \overline{h} = \frac{\overline{\text{Nu}}_Lk}{L}\nonumber
\end{equation}
where $\overline{Nu}_L$ is the surface averaged Nusselt number. Note that we evaluate all coolant properties at the film temperature
\begin{equation}
   \begin{split}
      T_f &= \frac{1}{2}(T_\infty + T_{cool})\nonumber\\
          &= 1100 \text{K}
   \end{split}
\end{equation}
So, our objective function is $\overline{\text{Nu}}_Lk$ (since $L$ is taken to be constant). Although it may reasonably be assumed that the flow is laminar, for the sake of generality, we use the following expression for the Nusselt number for flow over a heated plate (as in the previous scoping calcualtions for the fuel centerline temperature) which is valid for general Rayleigh number
\begin{equation}
    \begin{split}
        \overline{\text{Nu}}_L &= \bigg[ 0.825 + \frac{0.387\text{Ra}_L^{1/6}}{[1 + (0.492/\text{Pr})^{9/16}]^{8/27}}\bigg]^2\nonumber
    \end{split}
\end{equation}
where
\begin{equation}
    \begin{split}
        \text{Pr} &= \frac{\alpha}{\nu}=\frac{\mu}{k/c_p}\nonumber
    \end{split}
\end{equation}
and
\begin{equation}
    \begin{split}
        \text{Ra}_L = \text{Gr}_L \text{Pr} = \frac{g \beta\rho (T_s-T_\infty)L^3}{\mu\alpha}\nonumber
    \end{split}
\end{equation}

In [24]:
# Calculation Parameters
endmembers = ['NaF', 'KF', 'ZrF4', 'CaF2']
T_f = 1100 # K
g = 9.8 # m/s^2
Delta_T = 200 # K
L = 5 # m
r = 4E-03 # m

# A wrapper that extracts the nominal values from thermofunctions, this is necessary for calculating the derivative using numdifftools
def extract_nominal_value(func):
    def wrapper(x):
        y_with_uncertainty = func(x)
        return y_with_uncertainty.nominal_value  # Extract the nominal value
    return wrapper

# A constraint function which ensures that sum(x) = 1
def constraint(x):
    return 1 - np.sum(x)

# Define the constraints in a format that `minimize` can use
cons = ({'type': 'ineq', 'fun': constraint})

guess = [0.25, 0.25, 0.25] # An initial guess salt composition
bounds = [(0.05, 1), (0.05, 1), (0.05, 1)]  # Each individual composition is also bounded between 0 and 1

# Define the objective function
def objective(x):
    x = np.append(x, 1-sum(x))
    salt = frozendict({endmember: x[i] for i, endmember in enumerate(endmembers)})
    try:
        rho = db.get_tp('density', salt)
        beta = (-1/rho(T_f)*Derivative(extract_nominal_value(rho), n=1)(T_f)).nominal_value
        rho = db.get_tp('density', salt)(T_f).nominal_value
        mu = db.get_tp('viscosity', salt)(T_f).nominal_value
        k = db.get_tp('thermal_conductivity', salt)(T_f).nominal_value
        c_p = db.get_tp('liquid_heat_capacity', salt)(T_f).nominal_value
        alpha = k/(rho*c_p)
        try:
            Pr = mu*c_p/k
            Ra = g*beta*rho*Delta_T*L**3/(mu*alpha)
            if Pr < 0 or Ra < 0:
                return 1E+03
            Nu = ( 0.825 + 0.387*Ra**(1/6)/( 1 + (0.492/Pr)**(9/16) )**(8/27) )**2
        except ZeroDivisionError:
            return 1E+03
    except OverflowError:
        return 1E+03

    return -2*np.pi*r*L*Nu*k/L*Delta_T

In [25]:
# Now minimize the objective function
result = minimize(
    objective, 
    guess, 
    method='trust-constr',  # Using a different optimization method
    jac='2-point',          # Finite difference approximation if you do not provide a gradient
    constraints=cons,
    bounds=bounds,
    options={'verbose': 3, 'gtol': 1e-6, 'xtol': 1e-6, 'maxiter': 1000}
)

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |   4   |   0   | -3.0660e+04 | 1.00e+00 | 8.16e+03 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   0   |
|   2   |   8   |   1   | -5.0862e+04 | 7.00e+00 | 8.35e+03 | 9.10e-02 | 1.00e+00 |  1.00e-01   |   2   |
|   3   |  12   |   4   | -5.8219e+04 | 9.04e+00 | 3.91e+03 | 1.16e-01 | 1.61e+04 |  1.00e-01   |   1   |
|   4   |  16   |   7   | -5.0864e+04 | 9.60e+00 | 7.45e+02 | 0.00e+00 | 6.72e+04 |  1.00e-01   |   1   |
|   5   |  20   |   9   | -5.2263e+04 | 9.60e+00 | 7.33e+02 | 0.00e+00 | 6.72e+04 |  1.00e-01   |   4   |
|   6   |  24   |  11   | -5.3270e+04 | 9.60e+00 | 6.92e+02 | 0.00e+00 | 6.72e+04 |  1.00e-01   |   4   |
|   7   |  28   |  13   | -5.3877e+04 | 9.60e+00 | 6.03e+02 | 0.00e+00 | 6.72e+04 |  1.00e-01   |   4   |
|   8   |  32   |  15   | -5.5399e+04 | 9.60e+

In [26]:
result

           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: -55564.24307936636
                 x: [ 1.608e-01  5.000e-02  7.892e-01]
               nit: 125
              nfev: 572
              njev: 143
              nhev: 0
          cg_niter: 127
      cg_stop_cond: 4
              grad: [-2.352e+05 -1.600e+05 -2.351e+05]
   lagrangian_grad: [-2.416e-02 -1.455e-11  2.416e-02]
            constr: [array([ 2.454e-10]), array([ 1.608e-01,  5.000e-02,  7.892e-01])]
               jac: [array([[-1.000e+00, -1.000e+00, -1.000e+00]]), array([[ 1.000e+00,  0.000e+00,  0.000e+00],
                           [ 0.000e+00,  1.000e+00,  0.000e+00],
                           [ 0.000e+00,  0.000e+00,  1.000e+00]])]
       constr_nfev: [572, 0]
       constr_njev: [0, 0]
       constr_nhev: [0, 0]
                 v: [array([-2.352e+05]), array([ 2.001e+00, -7.519e+04, -5.877e-01])]
            method: tr_interior_point
      

In [27]:
objective([0.25, 0.25, 0.25])

-30659.672790395336

In [30]:
objective([1.608e-01, 5.000e-02,  7.892e-01])

-55564.24314818189